# Clasificador Olivo/gramíneas a partir de una red pre-entrenada:

In [2]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import  Convolution2D, MaxPooling2D, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from keras import applications
K.clear_session()

Using TensorFlow backend.


### Modelo VGG16:

Se carga el modelo VGG16: 

keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

Este modelo contiene los pesos obtenidos con el pre-entrenamiento en ImageNet. Por defecto, establece una imagen de entrada de 224x224.

In [4]:
vgg16 = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(480, 640, 3), classes=2)#Cargamos la red
vgg16.summary()#Muestra el contenido de la red.

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 480, 640, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 480, 640, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 480, 640, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 240, 320, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 240, 320, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 240, 320, 128)     147584    
_________________________________________________________________
bloc

Se ha pasado como argumento include_top=False. La red vgg16 que aporta keras contiene por defecto una última capa de predicción de mil neuronas (para clasificar mil clases diferentes). La red que se busca solo clasificará dos clases diferentes. Con include_top=False se elimina esta última capa. En la siguiente línea de código se carga la red al completo:

### Modificación de la última capa:

Se diseña la capa de salida para realizar la clasificación. Consytará de:

Capa GlobalAveragePooling2D.

Capa tipo Dense de 1000 neuronas y activación rectificador (relu)

In [ ]:
# add a global spatial average pooling layer
x = vgg.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(300, activation='relu')(x)
# and a logistic layer -- let's say we have 3 classes
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=predictions)

In [ ]:
model.summary()

### Congelación de capas:

Se ha cargado la red pre-entrenada vgg16, con los pesos de Imagenet. De esta forma las capas convolucionales y pooling que la forman actuarán como diferenciadoras de características, que serán usadas por la capa final que se ha incluido para realizar la clasificación.

Por tanto, se va a usar una red que ya ha 'aprendido' a clasificar en otros problemas, y se va a adaptar para el problema que aquí se trata (TRANSFER LEARNING).

Es por esto que no es necesario entrenar todas las capas de la red, y solo se entrenarán las 3 capas que se han añadido, congelando las demás.

In [5]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [6]:
for layer in model.layers[:19]:
   layer.trainable = False
for layer in model.layers[19:]:
   layer.trainable = True

In [7]:
model.summary()

Se observa que antes los parámetros no entrenables eran nulos, y ahora han pasado a ser 14,714,688.

### Tratamiento de las imágenes:

Se necesita adaptar el conjunto de imágenes para poder ser tratado por el modelo. Para ello se van a usar las funciones flow_from_directory() y ImageDataGenerator().

In [12]:
#Se almacenan en variables los directorios en los que se encuentran las imágenes
data_entrenamiento = './data_polen/entrenamiento'
data_validacion = './data_polen/validacion'

#Parámetros importantes:
epocas=10
longitud, altura = 640, 480
batch_size = 18 #Imágenes a procesar en cada paso
pasos = 10
validation_steps = 8 #Imágenes de validación que se pasan al final de cada época
clases = 2
lr = 0.00085 #Learning rate

flow_from_directory(directory, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, save_to_dir=None, save_prefix='', save_format='png', follow_links=False, subset=None, interpolation='nearest')

Con la función flow_from_directory() se pre-procesan las imágenes que se encuentran en los directorios previamente declarados. Además se le puede pasar como parámetros el tamaño al que se redimensionan las imágenes, o el algoritmo de interpolación.

keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0, height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0, fill_mode='nearest', cval=0.0, horizontal_flip=False, vertical_flip=False, rescale=None, preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None)

Gracias a la función ImageDataGenerator() se aplica al set de entrenamiento mecanismos de DATA ARGUMENTATION como inclinar, hacer zoom o invertir las imágenes.

In [13]:
###Procesamiento del conjunto de entrenamieto:
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, 
    shear_range=0.2, #Inclina las imágenes
    zoom_range=0.2, #Zoom a algunas imágenes
    horizontal_flip=True) #Invierte imágenes para distinguir direcionalidad

###Procesamiento del conjunto de validación:
#No es necesario inclinar, hacer zoom ni invertir las imágenes.
test_datagen = ImageDataGenerator(rescale=1. / 255)

###Generación del conjunto de entrenamieto:
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical') #Se busca una clasificación categórica

###Generación del conjunto de validación:
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

print(entrenamiento_generador.class_indices)

Found 54 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
{'gramineas': 0, 'olivo': 1}


### Definición del modelo CNN: función pérdida y optimizador:

In [14]:
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])

### Entrenamiento del modelo:

fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

In [15]:
model.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=len(entrenamiento_generador),
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
3/3 [==============================] - 587s 196s/step - loss: 0.7519 - acc: 0.5926 - val_loss: 0.6549 - val_acc: 0.5000
Epoch 2/10
3/3 [==============================] - 591s 197s/step - loss: 0.6557 - acc: 0.5741 - val_loss: 0.5965 - val_acc: 0.8500
Epoch 3/10
3/3 [==============================] - 598s 199s/step - loss: 0.5818 - acc: 0.9259 - val_loss: 0.5805 - val_acc: 0.6500
Epoch 4/10
3/3 [==============================] - 598s 199s/step - loss: 0.5165 - acc: 0.7407 - val_loss: 0.5756 - val_acc: 0.6500
Epoch 5/10
3/3 [==============================] - 595s 198s/step - loss: 0.4921 - acc: 0.6852 - val_loss: 0.5441 - val_acc: 0.6500
Epoch 6/10
3/3 [==============================] - 593s 198s/step - loss: 0.4563 - acc: 0.8704 - val_loss: 0.4522 - val_acc: 0.8500
Epoch 7/10
3/3 [==============================] - 589s 196s/step - loss: 0.4071 - acc: 0.9444 - val_loss: 0.4071 - val_acc: 0.9500
Epoch 8/10
3/3 [===================

### Cálculo de la función pérdida:

El método evaluate_generator(generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0) se encarga de calcular la función de pérdida dados unos datos de entrada y el nivel de aciertos del modelo para una muestra dada.

In [ ]:
score = model.evaluate_generator(validacion_generador, steps=pasos, verbose=1)
print('Test accuracy:', score[1])

In [ ]:
print(score)

### Guardar el modelo entrenado:

Para no tener que entrenar la red neuronal creada cada vez que se quiera usar, se crea un archivo donde se guarda el modelo creado, y otro donde se guardan los pesos obtenidos para las neuronas después del entranmiento.

In [16]:
import os 
dir = './vgg16_OliGram_Prueba/'
if not os.path.exists(dir):
    os.mkdir(dir)
model.save('./vgg16_OliGram_Prueba/modelo_vgg16.h5')#Se guarda la estructura de la cnn
model.save_weights('./vgg16_OliGram_Prueba/pesos_vgg16.h5')#Se guardan los pesos de la cnn

### Pruebas de clasificación:

In [17]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

La función load_image() transforma de forma interna las imágenes, tomando como argumento las dimensiones que admiten las arquitecturas implementadas y un método de interpolación. Se recomienda usar métodos de interpolación como bicubic o lanczos, frente a nearest que viene por defecto.

In [20]:
###Función predicción:
def predict(file):
  x = load_img(file, target_size=(altura, longitud))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0) #Zero mean pre-processing, normalize data.
  array = model.predict(x)
  print(array)  
  result = array[0]
  print(result)
  answer = np.argmax(result)
  print(answer)  
  if answer == 0:
    print("pred: Gramineas")
  elif answer == 1:
    print("pred: Olivo")
  return answer

In [21]:
predict('olivo1.jpg') #Imagen de olivo

[[0.41358513 0.5864149 ]]
[0.41358513 0.5864149 ]
1
pred: Olivo


1

In [22]:
predict('graminea2.jpg') #Imagen de graminea

[[9.9967051e-01 3.2954017e-04]]
[9.9967051e-01 3.2954017e-04]
0
pred: Gramineas


0

In [23]:
predict('graminea4.jpg') #Imagen de graminea

[[0.9773102  0.02268984]]
[0.9773102  0.02268984]
0
pred: Gramineas


0

In [24]:
predict('olivo5.jpg') #Imagen de olivo

[[0.01226435 0.9877357 ]]
[0.01226435 0.9877357 ]
1
pred: Olivo


1

In [31]:
predict('olivo2.jpg') #Imagen de olivo

[[0.4479748 0.5520252]]
[0.4479748 0.5520252]
1
pred: Olivo


1